In [1]:
import pandas as pd

from pathlib import Path
from itertools import chain

from tensorboard.backend.event_processing.event_accumulator import (
    EventAccumulator,
)

In [2]:
def get_results(path):
    """
    Индексы и значения лучших и худших точностей всех
    пользователей(кроме пропущенного) и пропущенного.
    """
    results_05 = {}
    for subj_log in (path / "logs").iterdir():
        subj_id = int(subj_log.stem.split("_")[1])
        with open(subj_log) as f:
            values_str = map(
                lambda x: str.strip(x).split("(")[-1][:-1].split("), "),
                f.readlines()[-4:],
            )
            values = map(lambda x: (float(x[0]), int(x[1])), values_str)
            results_05[subj_id] = dict(
                zip(
                    [
                        "worst_common_rate",
                        "worst_common_step",
                        "best_common_rate",
                        "best_common_step",
                        "worst_single_rate",
                        "worst_single_step",
                        "best_single_rate",
                        "best_single_step",
                    ],
                    chain.from_iterable(values),
                )
            )
    return pd.DataFrame(results_05).transpose().convert_dtypes().sort_index()

In [3]:
def extend_results(df, path):
    """
    Индексы и значения лучших и худших точностей всех
    пользователей(кроме пропущенного) и пропущенного.
    """
    add = {}
    dirs_with_data = filter(lambda x: x.stem != "logs", path.iterdir())
    for path in dirs_with_data:
        logdir = path / "hist"
        subj = int(path.stem.split("_", 3)[2])
        event_file = next(filter(Path.is_file, logdir.iterdir()))

        epoches = df.loc[subj][
            [
                "best_common_step",
                "worst_common_step",
                "best_single_step",
                "worst_single_step",
            ]
        ].astype(int)

        event_acc = EventAccumulator(str(event_file))
        event_acc.Reload()
        common = list(
            filter(
                lambda x: (
                    x.step == epoches.best_single_step
                    or x.step == epoches.worst_single_step
                ),
                event_acc.Scalars("Accuracy/train"),
            )
        )
        common_with_best_single, common_with_worst_single = (
            common
            if common[0].step == epoches.best_single_step
            else reversed(common)
        )
        single = list(
            filter(
                lambda x: (
                    x.step == epoches.best_common_step
                    or x.step == epoches.worst_common_step
                ),
                event_acc.Scalars(f"Accuracy/test_{subj}"),
            )
        )
        single_with_best_common, single_with_worst_common = (
            single
            if single[0].step == epoches.best_common_step
            else reversed(single)
        )
        add[subj] = {
            "common_with_best_single": common_with_best_single.value,
            "common_with_worst_single": common_with_worst_single.value,
            "single_with_best_common": single_with_best_common.value,
            "single_with_worst_common": single_with_worst_common.value,
        }
    df = df.merge(
        pd.DataFrame(add).transpose().sort_index(),
        left_index=True,
        right_index=True,
    ).round(4)
    return df[
        [
            "worst_common_step",
            "worst_common_rate",
            "single_with_worst_common",
            "best_common_step",
            "best_common_rate",
            "single_with_best_common",
            "worst_single_step",
            "worst_single_rate",
            "common_with_worst_single",
            "best_single_step",
            "best_single_rate",
            "common_with_best_single",
        ]
    ]

In [4]:
path_05 = Path("NetUpDownCoder3_3_ASGD_lr_1e-05")
df_05 = extend_results(get_results(path_05), path_05)

In [5]:
df_05

,worst_common_step,worst_common_rate,single_with_worst_common,best_common_step,best_common_rate,single_with_best_common,worst_single_step,worst_single_rate,common_with_worst_single,best_single_step,best_single_rate,common_with_best_single
2,4,0.7358,0.6880,56,0.9073,0.6735,79,0.4373,0.8396,27,0.7726,0.8675
3,6,0.797,0.9946,73,0.9063,0.8828,13,0.7302,0.8399,6,0.9946,0.7970
4,2,0.6151,0.9121,51,0.8919,0.8501,1,0.6176,0.7957,5,0.9147,0.7195
5,1,0.6637,1.0000,32,0.915,0.6250,13,0.5671,0.8173,1,1.0,0.6637
6,6,0.7049,0.9070,37,0.8927,0.9093,9,0.6465,0.7630,59,0.9605,0.8800
7,3,0.7582,0.8493,75,0.9,0.8242,48,0.79,0.8784,4,0.9429,0.8642
8,9,0.6544,0.5858,84,0.9003,0.9313,9,0.5858,0.6544,39,0.985,0.8621
9,32,0.7597,0.7081,38,0.9154,0.7712,19,0.5643,0.8585,43,0.8257,0.8907
10,2,0.7276,0.3961,66,0.8972,0.4573,1,0.3961,0.8281,19,0.8422,0.8457
11,14,0.6624,0.8810,43,0.8913,0.9876,31,0.8082,0.7567,2,1.0,0.7399


In [6]:
path_06 = Path("NetUpDownCoder3_3_ASGD_lr_1e-06")
df_06 = extend_results(get_results(path_06), path_06)

In [7]:
df_06

,worst_common_step,worst_common_rate,single_with_worst_common,best_common_step,best_common_rate,single_with_best_common,worst_single_step,worst_single_rate,common_with_worst_single,best_single_step,best_single_rate,common_with_best_single
2,37,0.7307,0.3848,39,0.9183,0.6676,37,0.3848,0.7307,28,0.7638,0.8904
3,6,0.7638,1.0000,36,0.9041,0.9019,2,0.6649,0.7793,6,1.0,0.7638
4,11,0.6906,0.8941,21,0.8823,0.8501,8,0.6434,0.7609,5,0.9225,0.7039
5,4,0.7845,0.7616,19,0.915,0.6134,6,0.5671,0.8233,4,0.7616,0.7845
6,6,0.6632,0.8744,45,0.8942,0.9349,3,0.6442,0.7899,41,0.9558,0.8614
7,4,0.7649,0.8744,31,0.8836,0.8836,23,0.79,0.8269,30,0.9498,0.8649
8,0,0.551,0.5365,29,0.8831,0.9335,0,0.5365,0.5510,34,0.9721,0.8336
9,2,0.7829,0.6797,34,0.9019,0.7996,7,0.6449,0.8653,40,0.8105,0.8877
10,2,0.6708,0.3961,21,0.9033,0.4654,2,0.3961,0.6708,9,0.847,0.8411
11,6,0.743,1.0000,45,0.8882,0.9947,7,0.8668,0.7513,1,1.0,0.8297
